In [ ]:
import pandas as pd
import lecilab_behavior_analysis.utils as utils
from pathlib import Path
import numpy as np
import matplotlib.pyplot as plt
import lecilab_behavior_analysis.df_transforms as dft
from sklearn.linear_model import LogisticRegression
import seaborn as sns
%load_ext autoreload
%autoreload 2


In [ ]:
# load data from cluster
tv_projects = utils.get_server_projects()
print(tv_projects)
# see the available animals
animals = utils.get_animals_in_project(tv_projects[1])
print(animals)
# download the data for a specific animal
mouse = "ACV007"
local_path = Path(utils.get_outpath()) / Path(tv_projects[1]) / Path("sessions") / Path(mouse)
# create the directory if it doesn't exist
local_path.mkdir(parents=True, exist_ok=True)
# download the session data
utils.rsync_session_data(
    project_name=tv_projects[1],
    animal=mouse,
    local_path=str(local_path),
    credentials=utils.get_idibaps_cluster_credentials(),
)
# load the data
df = pd.read_csv(local_path / Path(f'{mouse}.csv'), sep=";")

In [ ]:
# reduce the dataset to the psychometric version of the task
# Otherwise, we would include a lot of "easy" trials that would bias the fit
df_test = df[df["current_training_stage"] == "TwoAFC_visual_hard"]

In [ ]:
np.log(-2)

calculate the "evidences" and the choices

In [ ]:
df_test['visual_stimulus_devi'] = df_test['visual_stimulus'].apply(lambda x: abs(round(eval(x)[0] / eval(x)[1], 4)))
# transform it to a log value, preserving the negative sign
df_test['visual_stimulus_devi'] = df_test['visual_stimulus_devi'].apply(lambda x: np.log(x) if x > 0 else -np.log(-x))

# This was good in order to make the fit work for both left and right choices!
df_test['visual_stimulus_devi'] = df_test.apply(
    lambda row: row['visual_stimulus_devi'] if row['correct_side'] == 'left' else -row['visual_stimulus_devi'],
    axis=1
)
df_test['visual_stimulus_diff'] = df_test['visual_stimulus'].apply(lambda x: abs(eval(x)[0] - eval(x)[1]))
df_test['visual_stimulus_diff'] = df_test.apply(
    lambda row: row['visual_stimulus_diff'] if row['correct_side'] == 'left' else -row['visual_stimulus_diff'],
    axis=1
)
df_test["visual_stimulus_diff_binned"] = df_test['visual_stimulus_diff'] // 0.1
# !!!!! This introduces a bug!! What would happen on the trials where the mouse has to go right? Which value would be used then?
# df_test['left_choice'] = np.where((df_test['correct_side'] == 'left') & (df_test['correct'] == True), 1, 0)

# What you want is a value that goes from 0 to 1, indicating the probability of a left choice.
# For this fits, we really don't care about the correct side, we just want to know if the mouse chose left or right.

# I realized that the way I was plotting this before was using the performance of the mouse and the trials difficulty,
# in order to infer back the probability of a left choice. But we can actually use something simpler and less confusing:

# I had already created a function in the df_transforms module, to get the first choice of a mouse so we can use it here
df_test = dft.add_mouse_first_choice(df_test)
# This creates the column "first_choice" that indicates "left" or "right" for each trial.

# Now we can transform this to 0 and 1, where 0 is right and 1 is left
df_test['left_choice_new'] = df_test['first_choice'].apply(lambda x: 1 if x == 'left' else 0)

# By the way I am naming columns weirdly, just so you can play around with the different solutions and see how they work.
# Once we have what we need, we should clean up the code and use more meaningful names.

In [ ]:
# Now we can fit the data and visualize the results
X = df_test['visual_stimulus_devi'].values.reshape(-1, 1)
y = df_test['left_choice_new'].values.astype(int)
model = LogisticRegression()
model.fit(X, y)

# Now we have a model that predicts the probability of a left choice based on ANY visual stimulus deviation (xs).
# For plotting, we can generate a range of values for the visual stimulus deviation
import numpy as np
xs = np.linspace(df_test['visual_stimulus_devi'].min(), df_test['visual_stimulus_devi'].max(), 100).reshape(-1, 1)
y_prob = model.predict_proba(xs)[:, 1]

# Plot the actual choices of the mouse
fig, ax = plt.subplots(figsize=(5, 5))
sns.pointplot(
    x='visual_stimulus_devi',
    y='left_choice_new',
    data=df_test,
    estimator=lambda x: np.mean(x),
    color='blue',
    markers='o',
    errorbar=("ci", 95),
    ax=ax,
    label='Observed Choices',
    native_scale= True,
    linestyles='',
)

# overlay the fitted logistic regression curve
ax.plot(xs, y_prob, color='red', label='Logistic Regression Fit')
ax.set_xlabel("Visual Stimulus Deviation")
ax.set_ylabel("Probability of Left Choice")
plt.title("Psychometric Curve")
plt.legend()
plt.show()



Nuo, you are right, the cell below fits the binned data, which is wrong. That is only good for plotting. Please fix

In [ ]:
# Now we can fit the data and visualize the results
X = df_test['visual_stimulus_diff_binned'].values.reshape(-1, 1)
y = df_test['left_choice_new'].values.astype(int)
model = LogisticRegression()
model.fit(X, y)

##Why we still use binned difference to fit the data and then interpolate to predict, instead of using the all of the values of differences?

# Now we have a model that predicts the probability of a left choice based on ANY visual stimulus deviation (xs).
# For plotting, we can generate a range of values for the visual stimulus deviation
import numpy as np
xs = np.linspace(df_test['visual_stimulus_diff_binned'].min(), df_test['visual_stimulus_diff_binned'].max(), 100).reshape(-1, 1)
y_prob = model.predict_proba(xs)[:, 1]

# Plot the actual choices of the mouse
fig, ax = plt.subplots(figsize=(5, 5))
sns.pointplot(
    x='visual_stimulus_diff_binned',
    y='left_choice_new',
    data=df_test,
    estimator=lambda x: np.mean(x),
    color='blue',
    markers='o',
    errorbar=("ci", 95),
    ax=ax,
    label='Observed Choices',
    native_scale= True,
    linestyles='',
)

# overlay the fitted logistic regression curve
ax.plot(xs, y_prob, color='red', label='Logistic Regression Fit')
ax.set_xlabel("Visual Stimulus Difference")
ax.set_ylabel("Probability of Left Choice")
plt.title("Psychometric Curve")
plt.legend()
plt.show()



In [ ]:
# split by difficulty
sns.pointplot(
    x='visual_stimulus_diff_binned',
    y='left_choice_new',
    data=df_test,
    estimator=lambda x: np.mean(x),
    markers='o',
    errorbar=("ci", 95),
    # native_scale= True,
    linestyles='',
    hue="difficulty"
)
plt.plot()

In [ ]:
import statsmodels.api as sm

# Add an interaction term between visual_stimulus_devi and visual_stimulus_diff
def interaction_calc(row):
    is_left = 1 if row['correct_side'] == 'left' else -1
    return row['visual_stimulus_devi'] * row['visual_stimulus_diff_binned'] * is_left
df_test['interaction_term'] = df_test.apply(interaction_calc, axis=1)


# Prepare the independent variables
X_multi = df_test[['visual_stimulus_devi', 'visual_stimulus_diff_binned', 'interaction_term']]
X_multi_const = sm.add_constant(X_multi)
y = df_test['left_choice_new'].values.astype(int)

# Fit the logistic regression model with multiple regressors
logit_model_multi = sm.Logit(y, X_multi_const).fit()

# Display the summary, which includes p-values for all regressors
print(logit_model_multi.summary())

In [ ]:
# It is interesting to compare the effects of the relative difference between the two visual stimuli,
# and the absolute difference between them.

# Maybe what we can do is to train another logistic regression model, adding as well the absolute difference
# between the two visual stimuli, and see how it affects the probability of a left choice.
# Do you know what I mean?

for i in df_test.groupby('visual_stimulus_devi'):
    df_i = i[1].sort_values(by='visual_stimulus_diff')
    X = df_i['visual_stimulus_diff'].values.reshape(-1, 1)
    y = df_i['left_choice_new'].values.astype(int)
    model = LogisticRegression()
    model.fit(X, y)
    y_pred = model.predict(X)
    y_prob = model.predict_proba(X)[:, 1]
    plt.plot(X, y_prob, label=f"Visual Stimulus Deviation: {i[0]}")
    plt.legend()
plt.xlabel("Visual Stimulus Difference")
plt.ylabel("Probability of Left Choice")

In [ ]:
# Create 2D plots to visualize the influence of each regressor
fig, axes = plt.subplots(1, 3, figsize=(18, 5), sharey=True)

# Generate a range of values for visual_stimulus_devi and visual_stimulus_diff
devi_range = np.linspace(df_test['visual_stimulus_devi'].min(), df_test['visual_stimulus_devi'].max(), 100)
diff_range = np.linspace(df_test['visual_stimulus_diff_binned'].min(), df_test['visual_stimulus_diff_binned'].max(), 100)

# Plot for visual_stimulus_devi
axes[0].plot(devi_range, logit_model_multi.predict(sm.add_constant(np.column_stack((devi_range, np.zeros_like(devi_range), np.zeros_like(devi_range))))), color='blue')
axes[0].set_title('Influence of Visual Stimulus Deviation')
axes[0].set_xlabel('Visual Stimulus Deviation')
axes[0].set_ylabel('Probability of Left Choice')

# Plot for visual_stimulus_diff
axes[1].plot(diff_range, logit_model_multi.predict(sm.add_constant(np.column_stack((np.zeros_like(diff_range), diff_range, np.zeros_like(diff_range))))), color='green')
axes[1].set_title('Influence of Visual Stimulus Difference')
axes[1].set_xlabel('Visual Stimulus Difference')

# Plot for interaction term
# interaction_range = devi_range * diff_range.mean()
# axes[2].plot(devi_range, logit_model_interaction.predict(sm.add_constant(np.column_stack((devi_range, np.full_like(devi_range, diff_range.mean()), interaction_range)))), color='red')
interaction_range = devi_range * diff_range
axes[2].plot(devi_range, logit_model_multi.predict(sm.add_constant(np.column_stack((np.zeros_like(diff_range), np.zeros_like(devi_range), interaction_range)))), color='red')
axes[2].set_title('Influence of Interaction Term')
axes[2].set_xlabel('Visual Stimulus Deviation')

plt.tight_layout()
plt.show()

Matrix format

In [ ]:
# let's use the absolute value of the lowest visual stimulus as a proxy for the brightness of the visual stimulus
df_test['visual_stimulus_lowest'] = df_test['visual_stimulus'].apply(lambda x: abs(eval(x)[0]) if eval(x)[0] < eval(x)[1] else abs(eval(x)[1]))
# create 10 bins for the absolute value of the lowest visual stimulus
min_value = df_test['visual_stimulus_lowest'].min()
max_value = df_test['visual_stimulus_lowest'].max()
bins = np.linspace(min_value, max_value, 11)
df_test['visual_stimulus_lowest_binned'] = pd.cut(df_test['visual_stimulus_lowest'], bins=bins, labels=[f"{b:.2f}" for b in bins[:-1]])
# create a pivot table with the visual stimulus deviation and absolute value of the lowest visual stimulus
pivot_table_abs = df_test.pivot_table(
    index='visual_stimulus_lowest_binned',
    columns='visual_stimulus_devi',
    values='left_choice_new',
    aggfunc='mean',
    observed=True
)
# plot the heatmap
plt.figure(figsize=(5, 5))
sns.heatmap(pivot_table_abs, cmap='coolwarm', annot=True, fmt=".2f", cbar_kws={'label': 'Probability of Left Choice'})
plt.xlabel("Visual Stimulus Deviation")
plt.ylabel("Absolute Value of Lowest Visual Stimulus")
plt.title("Heatmap of Probability of Left Choice")
plt.show()

Multiple animals analysis


In [ ]:
df_dic = {}
for mouse in animals:
    local_path = Path(utils.get_outpath()) / Path(tv_projects[1]) / Path("sessions") / Path(mouse)
    # create the directory if it doesn't exist
    local_path.mkdir(parents=True, exist_ok=True)
    # download the session data
    utils.rsync_session_data(
        project_name=tv_projects[1],
        animal=mouse,
        local_path=str(local_path),
        credentials=utils.get_idibaps_cluster_credentials(),
    )
    # load the data
    df_dic[mouse] = pd.read_csv(local_path / Path(f'{mouse}.csv'), sep=";")

In [ ]:
df_dic_hard = {}
for df_name, df in zip(df_dic.keys(), df_dic.values()):
    if 'TwoAFC_visual_hard' in df["current_training_stage"].unique():
        df = df.dropna(subset = ['visual_stimulus'])
        df = df[df["current_training_stage"] == "TwoAFC_visual_hard"]

        df['visual_stimulus_devi'] = df['visual_stimulus'].apply(lambda x: abs(round(eval(x)[0] / eval(x)[1], 4)))
        df['visual_stimulus_devi'] = df.apply(
            lambda row: row['visual_stimulus_devi'] if row['correct_side'] == 'left' else -row['visual_stimulus_devi'],
            axis=1
        )
        df['visual_stimulus_diff'] = df['visual_stimulus'].apply(lambda x: abs(eval(x)[0] - eval(x)[1]))
        df['visual_stimulus_diff'] = df.apply(
            lambda row: row['visual_stimulus_diff'] if row['correct_side'] == 'left' else -row['visual_stimulus_diff'],
            axis=1
        )
        df["visual_stimulus_diff_binned"] = df['visual_stimulus_diff'] // 0.1
        df = dft.add_mouse_first_choice(df)
        df['left_choice_new'] = df['first_choice'].apply(lambda x: 1 if x == 'left' else 0)
        
        df_dic_hard[df_name] = df

In [ ]:
df_devi_diffBin_inter_p = pd.DataFrame()
df_devi_diffBin_inter_coef = pd.DataFrame()
for df_name, df in zip(df_dic_hard.keys(), df_dic_hard.values()):
    df['interaction_term'] = df.apply(interaction_calc, axis=1)
    # Prepare the independent variables
    X_multi = df[['visual_stimulus_devi', 'visual_stimulus_diff_binned', 'interaction_term']]
    X_multi_const = sm.add_constant(X_multi)
    y = df['left_choice_new'].values.astype(int)

    # Fit the logistic regression model with multiple regressors
    logit_model_multi = sm.Logit(y, X_multi_const).fit()

    df_devi_diffBin_inter_p[df_name] = logit_model_multi.pvalues
    df_devi_diffBin_inter_coef[df_name] = logit_model_multi.params

In [ ]:
df_devi_diff_inter_p = pd.DataFrame()
df_devi_diff_inter_coef = pd.DataFrame()
for df_name, df in zip(df_dic_hard.keys(), df_dic_hard.values()):
    df['interaction_term'] = df.apply(interaction_calc, axis=1)
    # Prepare the independent variables
    X_multi = df[['visual_stimulus_devi', 'visual_stimulus_diff', 'interaction_term']]
    X_multi_const = sm.add_constant(X_multi)
    y = df['left_choice_new'].values.astype(int)

    # Fit the logistic regression model with multiple regressors
    logit_model_multi = sm.Logit(y, X_multi_const).fit()

    df_devi_diff_inter_p[df_name] = logit_model_multi.pvalues
    df_devi_diff_inter_coef[df_name] = logit_model_multi.params

In [ ]:
df_devi_diffBin_inter_p.rename(index={'visual_stimulus_devi': 'devi', 'visual_stimulus_diff_binned': 'diff', 'interaction_term': 'inter'}, inplace=True)
df_devi_diff_inter_p.rename(index={'visual_stimulus_devi': 'devi', 'visual_stimulus_diff': 'diff', 'interaction_term': 'inter'}, inplace=True)
for df_name, color in zip(df_dic_hard.keys(), sns.color_palette("colorblind", len(df_dic_hard))):
    plt.plot (df_devi_diff_inter_p[df_name], label=df_name+ 'devi_diff_inter', color=color)
    plt.plot (df_devi_diffBin_inter_p[df_name], label=df_name+ 'devi_diffBin_inter', color=color, linestyle='--')
plt.axhline(y=0.05, color='k', linestyle='--', label='p-value threshold')
plt.xlabel("Regressors")
plt.ylabel("p-value")
plt.legend(loc = (1 , 0))

In [ ]:
df_devi_diffBin_inter_coef.rename(index={'visual_stimulus_devi': 'devi', 'visual_stimulus_diff_binned': 'diff', 'interaction_term': 'inter'}, inplace=True)
df_devi_diff_inter_coef.rename(index={'visual_stimulus_devi': 'devi', 'visual_stimulus_diff': 'diff', 'interaction_term': 'inter'}, inplace=True)
for df_name, color in zip(df_dic_hard.keys(), sns.color_palette("colorblind", len(df_dic_hard))):
    plt.plot (df_devi_diff_inter_coef[df_name], label=df_name+ 'devi_diff_inter', color=color)
    plt.plot (df_devi_diffBin_inter_coef[df_name], label=df_name+ 'devi_diffBin_inter', color=color, linestyle='--')
plt.xlabel("Regressors")
plt.ylabel("Coefficient")
plt.legend(loc = (1 , 0))